In [ ]:
# ==============================================
# CHAPTER: Survival Analysis
# ==============================================

# ===============================
# 문제 0) 라이브러리 설치
# - lifelines 라이브러리를 설치한 후, pandas를 불러옵니다.
# ===============================
!pip install lifelines -q
import pandas as pd

# ===============================
# 문제 1) 데이터 불러오기
# - "example.csv" 파일을 데이터프레임으로 불러와 data_frame 변수에 저장
# ===============================
data_frame = pd.read_csv("example.csv")

# ===============================
# 문제 2) 생존분석 기초 조사
# ===============================
# 2-1) 생존함수 (Survival Function, S(t))
#   - 시간 t 이후에도 사건(Event)이 발생하지 않을 확률을 나타냄
#   - 수식: S(t) = P(T > t)
#
# 2-2) 위험함수 (Hazard Function, h(t))
#   - 시간 t에서 사건이 '즉시' 발생할 위험률(조건부 확률)
#   - 수식: h(t) = f(t) / S(t)
#
# 2-3) Kaplan-Meier 추정 (Kaplan-Meier Estimator)
#   - 비모수적 방식으로 생존함수 추정
#   - 추정식: Ŝ(t) = ∏ (1 - d_i / n_i) (t_i ≤ t)
#   - 특징: 검열데이터 허용, 분포 가정 없음, 계단형 생존곡선 생성

# ===============================
# 문제 3) Kaplan-Meier 생존곡선 추정 및 시각화
# ===============================
from lifelines import KaplanMeierFitter
import matplotlib.pyplot as plt

# Kaplan-Meier 모델 적합
kmf = KaplanMeierFitter()
kmf.fit(
    durations=data_frame['duration'],
    event_observed=data_frame['event'],
    label='Survival Probability'
)

# step-plot으로 생존곡선 시각화
fig, ax = plt.subplots(figsize=(8, 5))
ax.step(
    kmf.survival_function_.index,
    kmf.survival_function_['Survival Probability'],
    where='post'
)
ax.set_xlabel('Time')
ax.set_ylabel('Survival Probability')
ax.set_title('Survival Curve')
ax.grid(True)
plt.tight_layout()
plt.show()

# ===============================
# 문제 4) 중앙생존시간(Median Survival Time)
# - S(t) = 0.5가 되는 시간 t' 계산 및 의미 해석
# ===============================
surv_df = kmf.survival_function_
t_prime = surv_df[surv_df['Survival Probability'] <= 0.5].index.min()
print(f"S(t) = 0.5가 되는 시간 t': {t_prime}")
# 의미: 모집단 절반이 사건을 경험하기 전까지 생존한 시간의 중앙값

# ===============================
# 문제 5) 브랜드별(KM) 생존곡선
# - 데이터프레임을 brand별로 나누어 Kaplan-Meier 생존곡선을 비교
# ===============================
blue_color = '#1f77b4'
orange_color = '#ff7f0e'
colors = [blue_color, orange_color]

fig, ax = plt.subplots(figsize=(8, 5))
fig.patch.set_facecolor('white')
ax.set_facecolor('white')
ax.grid(True, color='lightgray', linestyle='--', linewidth=0.7, alpha=0.7)

for (name, group), color in zip(data_frame.groupby('brand'), colors):
    kmf = KaplanMeierFitter()
    kmf.fit(
        durations=group['duration'],
        event_observed=group['event'],
        label=name
    )
    kmf.plot_survival_function(ax=ax, ci_show=True, color=color)

ax.set_title("Survival curves by brand")
ax.set_xlabel("timeline")
ax.set_ylabel("")  
ax.legend(title="brand")
plt.tight_layout()
plt.show()

# ===============================
# 문제 6) 로그랭크 검정(Log-rank test)
# - 그룹 간 생존함수 차이 통계적 유의성 검정
# ===============================
from lifelines.statistics import logrank_test

brands = data_frame['brand'].unique()
b1, b2 = brands[0], brands[1]

group1 = data_frame.query("brand == @b1")
group2 = data_frame.query("brand == @b2")

result = logrank_test(
    durations_A=group1['duration'].values,
    durations_B=group2['duration'].values,
    event_observed_A=group1['event'].values,
    event_observed_B=group2['event'].values
)

print("p-value:", result.p_value)
if result.p_value < 0.05:
    print("유의수준 < 0.05: 전반적인 차이 존재")
else:
    print("유의수준 >= 0.05: 전반적인 차이 부재")

# ===============================
# 문제 7) Robusta 측 주장 평가
# ===============================
# 시각적 관찰: Robusta 곡선이 Cheapz 위에 위치하나, 말단부에서 신뢰구간 겹침
# 의미: 겹치는 구간에서는 통계적으로 유의하지 않을 가능성 존재
# 결론: 말단부 제외 시 시각적 우위 가능, 하지만 통계검정 없이 'Robusta가 항상 안전하다' 단정 불가


In [ ]:
# ==============================================
# CHAPTER 2: 프로젝트 성공 분석 (문제 0~10 + BONUS)
# ==============================================

# ===============================
# 문제 0) 데이터 불러오기
# ===============================
import pandas as pd

df1 = pd.read_csv('history_static.csv')
df1.head()
df1.info()

# ===============================
# 문제 1) Status와 day_succ 간 관계 히스토그램
# ===============================
import matplotlib as mpl
import matplotlib.pyplot as plt

mpl.rc('font', family='Malgun Gothic')
mpl.rc('axes', unicode_minus=False)

fig, ax = plt.subplots(figsize=(10, 5))
bins = list(range(0, 65, 5))
success = df1.loc[df1['Status'] == 1, 'day_succ']
failure = df1.loc[df1['Status'] == 0, 'day_succ']

ax.hist(
    [success, failure],
    bins=bins,
    label=['성공 (Status=1)', '실패 (Status=0)'],
    color=['skyblue', 'salmon'],
    edgecolor='black',
    alpha=0.6
)
ax.set_title('day_succ 분포')
ax.set_xlabel('day_succ (일)')
ax.set_ylabel('프로젝트 수')
ax.set_xticks(bins)
ax.legend()
ax.grid(True)
plt.tight_layout()
plt.show()

# ===============================
# BONUS) 성공한 프로젝트 특징 인사이트 + 결론
# ===============================
# 1. 완료 시간 분포: 성공 프로젝트 day_succ는 0~30일에 집중
# 2. 빠른 완료 경향: 0~5, 10~20일 구간에도 성공 사례 존재
# 3. 장기화 시 실패율 증가: 35일 이후 성공 사례 급감, 60일 부근 실패 급증
# 4. 임계점: 약 30일 전후 교차, 30일 안에 완료하지 못하면 실패 위험 상승
# 결론: 20~25일 내 완료 역량과 30일 마감 준수가 성공률 제고에 중요

# ===============================
# 문제 2) Kaplan-Meier 생존함수 추정
# ===============================
from lifelines import KaplanMeierFitter

kmf = KaplanMeierFitter()
kmf.fit(durations=df1['day_succ'], event_observed=df1['Status'])

fig, ax = plt.subplots(figsize=(8, 5))
kmf.survival_function_.plot(ax=ax, drawstyle='steps-post')
ax.set_title("전체 생존함수 추정")
ax.set_xlabel("Time (day_succ)")
ax.set_ylabel("KM_estimate")
ax.grid(True)
plt.tight_layout()
plt.show()

# ===============================
# 문제 3) 누적 성공 확률 그래프 (1 - S(t)), x축 로그 스케일
# ===============================
sf_col = kmf.survival_function_.columns[0]
sf_vals = kmf.survival_function_[sf_col]
cum_success = 1 - sf_vals
times = sf_vals.index

fig, ax = plt.subplots(figsize=(8, 5))
ax.plot(times, cum_success, drawstyle='steps-post', color='darkorange', label='1 - S(t)')
ax.set_xscale('log')
ax.set_xlabel("Days (log scale)")
ax.set_ylabel("Probability of Success by Day t")
ax.set_title("Cumulative Success Probability (1-S(t))")
ax.grid(which='both', linestyle='--', linewidth=0.5)
ax.legend()
plt.tight_layout()
plt.show()

# ===============================
# 문제 4) 로지스틱 분포 가정 타당성 평가 (주석)
# ===============================
# 1. (1-S(t))와 로지스틱 CDF 유사성
#    - Kaplan-Meier 결과에서 1-S(t)는 t까지 성공할 누적확률(CDF 역할)
#    - 로지스틱 CDF: F(t) = 1 / (1 + exp(-(t - μ)/s)), S자형 곡선
#    - 초기 완만 → 중간 급격 → 후기 포화 형태, 크라우드펀딩 누적 후원 패턴과 유사
#
# 2. 크라우드펀딩 도메인 고려
#    - 초기 집중 기여: 프로젝트 론칭 직후 후원금 빠르게 모임
#    - 성숙 단계 포화: 목표액 근접 시 후원 속도 둔화
#    - 로지스틱 CDF는 초기 완만 → 중간 급격 → 후기 포화 자연스럽게 모델링 가능
#
# 3. 결론
#    - 크라우드펀딩의 누적 성공 확률이 전형적인 S자형을 보인다면,
#      로지스틱 분포 가정은 합리적임

# ===============================
# 문제 5) 로그-로지스틱 AFT 모형 적합
# ===============================
from lifelines import LogLogisticAFTFitter

reg1 = LogLogisticAFTFitter()
reg1.fit(df1, duration_col='day_succ', event_col='Status')
reg1.print_summary()

# ===============================
# 문제 6) 동적 특성 반영 데이터 병합
# ===============================
temporal = pd.read_csv('history_temporal.csv')
df2 = pd.concat([df1, temporal], axis=1)
df2.head()
df2.info()

# 주석: 동적 특성 반영 이유
# - 초기 며칠 간 후원 양상(동적 특성)은 프로젝트 성공/실패를 예측하는 중요한 힌트
# - 단순 정적 데이터만으로는 프로젝트 성공률을 충분히 설명하기 어려움
# - df2에는 정적 변수(df1)와 초기 후원 동적 변수(temporal)가 모두 포함되어
#   AFT 모델에서 보다 정확한 조건부 생존 확률 추정 가능

# ===============================
# 문제 7) df2 기반 로그-로지스틱 AFT 모형 적합
# ===============================
reg2 = LogLogisticAFTFitter()
reg2.fit(df2, duration_col='day_succ', event_col='Status')
reg2.print_summary()

# ===============================
# 주석: 확장 데이터셋 활용 이유
# ===============================
# - df2에는 정적 특성(df1)과 초기 동적 후원 특성(temporal)이 모두 포함됨
# - 초기 후원 동적 특성이 프로젝트 성공/실패(day_succ, Status)에 영향을 줄 수 있어,
#   모델 적합 시 예측력이 향상될 가능성이 있음
# - 이전 문제(reg1)에서는 정적 특성만 사용했으므로, reg2를 통해 동적 정보 활용 여부 확인 가능

# ===============================
# 문제 8) reg1, reg2 모델 AIC 비교 및 시사점
# ===============================
# 로그-로지스틱 AFT 모형 AIC 비교
# 모델      AIC 값        특성
# reg1   83,778.82   정적 변수만 사용한 기본 베이스라인 구조
# reg2   75,416.70   정적 + 시계열 동적 지표를 결합한 확장형 모델

# 모델 선택 기준:
# - AIC 값이 낮을수록 설명력과 예측력이 우수
# - 따라서 reg2가 reg1 대비 더 적합도가 높다고 판단

# 추가 성능 지표:
# - reg2의 Concordance index = 0.90 → 높은 예측 정확도 확인

# 시사점:
# - 동적 지표를 포함함으로써 크라우드펀딩 성공 시점 예측의 정밀도가 크게 향상
# - 정적 특성만으로는 포착하기 어려운 시간 의존적 요인의 중요성을 강조

# ===============================
# BONUS 문제 9) Concordance Index(C-Index) 해석
# ===============================
# Concordance Index (C-Index) 개념
# - 정의:
#   생존 모델이 서로 다른 두 개체의 관측된 생존 시간 순서를 얼마나 올바르게 예측하는지 나타내는 지표
#   Harrell’s C 또는 C-Index라고도 불림
#   회귀모델에서의 AUC 개념에 대응
#
# - 해석 가이드라인:
#   C-Index 값     해석
#   1.0           모든 샘플 쌍의 순서를 완벽히 맞춤 (이상적 예측)
#   0.5           무작위 수준, 예측 능력 없음
#   >0.7           일반적으로 충분히 우수한 예측 성능으로 간주
#
# - reg2 모델 사례:
#   C-Index = 0.90
#   실제로 두 프로젝트 중 더 오래 지속된 쪽을 90% 확률로 정확히 예측함을 의미
#
# - 비교:
#   C-Index가 0.5였다면 → 순서 구분 불가, 예측력 없음
#   0.90 수준 → 프로젝트 지속 시간 순위 예측이 매우 정밀
#
# - 결론:
#   높은 C-Index 값은 선택된 AFT 모형이 관측된 생존 시간의 상대적 순위를
#   탁월하게 식별하고 있음을 보여 줌

# ===============================
# 문제 10) 선택된 reg2 모델 계수 해석
# ===============================
# 관심 변수: [동적 정보] 3f_plg, [정적 정보] duration, has_video
#
# reg2 모형 계수 해석
#
# 1. 3f_plg (3일차 누적 모금액)
#    - 계수: -1.10
#    - 해석: 3일차까지 모금액이 많을수록 성공 시점이 앞당겨짐
#      → 초기 후원 속도가 빠른 프로젝트일수록 목표 달성까지 걸리는 시간이 짧아짐
#
# 2. duration (펀딩 모집 기간)
#    - 계수: +0.35
#    - 해석: 모집 기간이 길수록 성공까지 소요되는 시간이 늘어남
#      → 더 긴 준비·홍보 기간은 프로젝트 목표 달성 시점을 뒤로 미룸
#
# 3. has_video (영상 포함 여부)
#    - 계수: -0.08
#    - 해석: 설명 페이지에 영상이 포함된 프로젝트는 성공 시점이 다소 앞당겨짐
#      → 비디오 콘텐츠가 후원자의 관심과 참여를 유도하여 목표 달성 속도를 높임
